In [ ]:
# Cluster statistics
# print("\n📈 Cluster Statistics:")
# cluster_stats = cluster_df.groupby('cluster').agg(
#     Size=('ticker', 'count'),
#     Avg_Correlation=('ticker', lambda x: corr_subset.loc[x,x].values.mean()),
#     Avg_Score=('score', 'mean'),
# ).reset_index().round(2)
print(cluster_stats[['cluster', 'Size', 'Avg_Correlation', 'Avg_Score']].to_string(index=False))

# deepseek
# Cluster statistics
# print("\n📈 Cluster Statistics:")
# cluster_stats = cluster_df.groupby('cluster').agg(
#     Size=('ticker', 'count'),
#     Avg_Correlation=('ticker', lambda x: corr_subset.loc[x,x].values.mean()),
#     Avg_Score=('score', 'mean'),
# ).reset_index().round(2)

# Create a dataframe to store the cluster statistics
cluster_stats_df = cluster_stats[['cluster', 'Size', 'Avg_Correlation', 'Avg_Score']].copy()

# Print the statistics
print(cluster_stats_df.to_string(index=False))

# google
# Cluster statistics
# print("\n📈 Cluster Statistics:")
# cluster_stats = cluster_df.groupby('cluster').agg(
#     Size=('ticker', 'count'),
#     Avg_Correlation=('ticker', lambda x: corr_subset.loc[x,x].values.mean()),
#     Avg_Score=('score', 'mean'),
# ).reset_index().round(2)

# Print the cluster statistics
print(cluster_stats[['cluster', 'Size', 'Avg_Correlation', 'Avg_Score']].to_string(index=False))

# Store the cluster statistics in a DataFrame (already done in the previous step)
cluster_stats_df = cluster_stats[['cluster', 'Size', 'Avg_Correlation', 'Avg_Score']]

In [ ]:
# Cluster statistics
print("\n📈 Cluster Statistics:")
cluster_stats = cluster_df.groupby('cluster').agg(
    Size=('ticker', 'count'),
    Avg_Correlation=('ticker', lambda x: corr_subset.loc[x,x].values.mean()),
    Avg_Score=('score', 'mean'),
).reset_index().round(2)
print(cluster_stats[['cluster', 'Size', 'Avg_Correlation', 'Avg_Score']].to_string(index=False))

# deepseek
# Cluster statistics
print("\n📈 Cluster Statistics:")
cluster_stats = cluster_df.groupby('cluster').agg(
    Size=('ticker', 'count'),
    Avg_Correlation=('ticker', lambda x: corr_subset.loc[x,x].values.mean()),
    Avg_Score=('score', 'mean'),
).reset_index().round(2)

# Create a dataframe to store the cluster statistics
cluster_stats_df = cluster_stats[['cluster', 'Size', 'Avg_Correlation', 'Avg_Score']].copy()

# Print the statistics
print(cluster_stats_df.to_string(index=False))

# google
# Cluster statistics
print("\n📈 Cluster Statistics:")
cluster_stats = cluster_df.groupby('cluster').agg(
    Size=('ticker', 'count'),
    Avg_Correlation=('ticker', lambda x: corr_subset.loc[x,x].values.mean()),
    Avg_Score=('score', 'mean'),
).reset_index().round(2)

# Print the cluster statistics
print(cluster_stats[['cluster', 'Size', 'Avg_Correlation', 'Avg_Score']].to_string(index=False))

# Store the cluster statistics in a DataFrame (already done in the previous step)
cluster_stats_df = cluster_stats[['cluster', 'Size', 'Avg_Correlation', 'Avg_Score']]

In [26]:
import sys
from pathlib import Path
import pandas as pd

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils

# SOURCE_PATH_OHLCV = '..\data\df_OHLCV_2025-03-07_clean.pkl'
# SOURCE_PATH_STOCK = '..\data\df_finviz_stocks_n_ratios.pkl'
# SOURCE_PATH_ETF = '..\data\df_finviz_etfs_n_ratios.pkl'

SOURCE_PATH, DEST_PATH = utils.main_processor(
    data_dir='..\data',  # search project ..\data
    downloads_dir=None,  # None searchs Downloads dir, '' omits search1
    downloads_limit=60,  # search the first 10 files
    clean_name_override='df_finviz.pkl',  # override filename
    start_file_pattern='df_finviz_2025', # search for files starting with 'df_'
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\python310.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py310\\stocks\\src', 'c:\\Users\\ping\\Files_win10\\python\\py310\\stocks\\src', 'c:\\Users\\ping\\Files_win10\\python\\py310\\stocks\\src', 'c:\\Users\\ping\\Files_win10\\python\\py310\\stocks\\src']


<span style='color:#00ffff;font-weight:500'>[Downloads] Scanned latest 60 files • Found 10 'df_finviz_2025' matches</span>

**Available 'df_finviz_2025' files:**

- (1) `[DOWNLOADS]` `df_finviz_2025-03-26.pkl` <span style='color:#00ffff'>(0.51 MB, 2025-03-26 15:12)</span>

- (2) `[DOWNLOADS]` `df_finviz_2025-03-25.pkl` <span style='color:#00ffff'>(0.51 MB, 2025-03-25 15:23)</span>

...

- (8) `[DOWNLOADS]` `df_finviz_2025-03-18.pkl` <span style='color:#00ffff'>(0.51 MB, 2025-03-18 13:38)</span>

- (9) `[DOWNLOADS]` `df_finviz_2025-03-17.pkl` <span style='color:#00ffff'>(0.46 MB, 2025-03-17 13:06)</span>

- (10) `[DOWNLOADS]` `df_finviz_2025-03-14.pkl` <span style='color:#00ffff'>(0.46 MB, 2025-03-14 15:30)</span>


Input a number to select file (1-10)



    **Selected paths:**
    - Source: `C:\Users\ping\Downloads\df_finviz_2025-03-14.pkl`  
    - Destination: `..\data\df_finviz.pkl`
    

In [27]:
import re

# Extract date using regex pattern
date_pattern = r'(\d{4}-\d{2}-\d{2})'
match = re.search(date_pattern, SOURCE_PATH)
if match:
  date_str = match.group(1)
  print(f"Extracted date: {date_str}")
else:
  print("No date found in the path")

Extracted date: 2025-03-14


In [28]:
# create_config.py
def create_config_file(date_str):
    """Create config.py with date_str as a string variable"""
    config_content = f"""# config.py
# Automatically generated date configuration
date_str = '{date_str}'  # Date in YYYY-MM-DD format
"""
    
    with open('config.py', 'w') as f:
        f.write(config_content)
    
    print(f"config.py created successfully with date: {date_str}")


In [29]:
create_config_file(date_str)

config.py created successfully with date: 2025-03-14


In [30]:
# script_using_config.py
from config import date_str

print(f"Date string from config: {date_str}")  # Direct string access

Date string from config: 2025-03-14
